In [ ]:
# import
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, LinearRegression

%matplotlib inline
import gc

In [ ]:
df = pd.read_csv('C:/Users/y/Desktop/424brooklyn/data/lintong/COS424_final/workingdata.csv')
# df = pd.read_csv('/home/yingyig/data/workingdata.csv')
df.head()

# pre-processing

In [ ]:
# get rid of missing sale_price and outliers
sale_prc_threshold = 10000  # to be determined
null_mask = (~df.sale_price.isnull()) & (df.sale_price >= sale_prc_threshold)
df_null_filtered = df[null_mask].reset_index(drop=True)
del df

# split training and test before pre-processing
frac = 0.25
ntest = int(len(df_null_filtered) * frac)
test_indices = [int(np.floor(len(df_null_filtered) * np.random.rand())) for i in range(ntest)] 
train_indices = list(set(df_null_filtered.index) - set(test_indices))
df_train = df_null_filtered.iloc[train_indices, :]
df_test = df_null_filtered.iloc[test_indices, :]
del df_null_filtered

# impute with column mean
df_train_imputed = df_train.fillna(df_train.mean())
df_test_imputed = df_test.fillna(df_test.mean())
del df_train
del df_test

# separate X and y
X_train = df_train_imputed.loc[:, df_train_imputed.columns != 'sale_price']
y_train = df_train_imputed.sale_price
X_test = df_test_imputed.loc[:, df_test_imputed.columns != 'sale_price']
y_test = df_test_imputed.sale_price
del df_train_imputed
del df_test_imputed

gc.collect()
#train_cor = df_train_imputed.corr()
#test_cor = df_test_imputed.corr()

### modeling

In [ ]:
# prepare models
models = []
models.append(('LR', LinearRegression()))
#models.append(('Lasso', Lasso()))
#models.append(('Ridge', Ridge()))
#models.append(('EN', ElasticNet()))
#models.append(('LassoCV', LassoCV()))
#models.append(('RidgeCV', RidgeCV()))
#models.append(('ENCV', ElasticNetCV()))

# evaluate each model in turn
names = []
for name, model in models:
    model.fit(X_train, y_train)
    print("\n%s --- %f" % (name, model.score(X_test, y_test)))  # adjusted r^2